# Seldon deployment of failure classification
In this notebook, we deploy a service for classifying flakes. We take the experiments in the [failure type classifier notebook](failure_type_classifier.ipynb) and train a sklearn pipeline with all the components, store the pipeline on S3, and deploy it using Seldon. Finally, we test the service for inference on an example request. 

In [1]:
import json
import os
import gzip
import datetime
import pandas as pd
import numpy as np
from dotenv import load_dotenv, find_dotenv
import warnings
from sklearn.pipeline import Pipeline

from ipynb.fs.defs.failure_type_functions import (
    print_report,
    detect_failures,
    normalize,
    format_results,
    CephCommunication,
    decode_run_length
)

load_dotenv(find_dotenv())
warnings.filterwarnings("ignore")

# Load Dataset

In [2]:
## Specify variables

METRIC_NAME = "failure_type"

# Specify the path for input grid data
INPUT_DATA_PATH = "../../data/raw/testgrid_810.json.gz"

# Specify the path for output metric data
OUTPUT_DATA_PATH = f"../../data/processed/metrics/{METRIC_NAME}"

## CEPH Bucket variables
## Create a .env file on your local with the correct configs
s3_endpoint_url = os.getenv("S3_ENDPOINT")
s3_access_key = os.getenv("S3_ACCESS_KEY")
s3_secret_key = os.getenv("S3_SECRET_KEY")
s3_bucket = os.getenv("S3_BUCKET")
s3_path = os.getenv("S3_PROJECT_KEY", "metrics")
s3_input_data_path = "raw_data"
s3_output_data_path = "failure_type"

# Specify whether or not we are running this as a notebook or part of an automation pipeline.
AUTOMATION = os.getenv("IN_AUTOMATION")

In [3]:
## Import data
timestamp = datetime.datetime.today()
filename = f"testgrid_{timestamp.day}{timestamp.month}.json"
# timestamp = datetime.datetime(2021, 4, 14)

if AUTOMATION:
    filename = f"testgrid_{timestamp.day}{timestamp.month}.json"
    cc = CephCommunication(s3_endpoint_url, s3_access_key, s3_secret_key, s3_bucket)
    s3_object = cc.s3_resource.Object(s3_bucket, f"{s3_input_data_path}/{filename}")
    file_content = s3_object.get()["Body"].read().decode("utf-8")
    testgrid_data = json.loads(file_content)

else:
    with gzip.open(INPUT_DATA_PATH, "rb") as read_file:
        testgrid_data = json.load(read_file)

In [4]:
selected_dashboard = '"redhat-openshift-ocp-release-4.3-informing"'
selected_job = "release-openshift-origin-installer-e2e-gcp-compact-4.3"

In [5]:
grid = testgrid_data[selected_dashboard][selected_job]["grid"]

In [6]:
x = np.array(list(pd.DataFrame(grid).statuses.apply(decode_run_length)))
x = pd.DataFrame(x).apply(lambda x: [normalize(y) for y in x])

In [7]:
results = detect_failures(testgrid_data, grid, selected_dashboard, selected_job)

In [8]:
results = format_results(results)
results

{'flaky_tests': '[(6, {(15, 19): 40.0, (23, 27): 40.0}, [(datetime.date(2020, 9, 22), datetime.date(2020, 9, 18)), (datetime.date(2020, 9, 14), datetime.date(2020, 9, 10))])]',
 'infra_flake': '[(datetime.date(2020, 10, 2), 1), (datetime.date(2020, 10, 1), 3)]',
 'install_flake': '[datetime.date(2020, 9, 2), datetime.date(2020, 8, 31)]',
 'new_test_failure': '[]'}

# Train SKlearn Pipeline

In [9]:
class FailClassifier:
    def __init__(self, testgrid_data, grid, selected_dashboard, selected_job):

        self.testgrid_data = testgrid_data
        self.grid = grid
        self.selected_dashboard = selected_dashboard
        self.selected_job = selected_job
        print(">>>>>>>>>>>init>>>>>>>>>>>")

    def fit(self):
        print(">>>>>>>>>>>fit>>>>>>>>>>>")

    def predict(self, testgrid_data, grid, selected_dashboard, selected_job):
        results = detect_failures(testgrid_data, grid, selected_dashboard, selected_job)
        results = format_results(results)
        return results

In [10]:
pipeline = Pipeline([("fail_classifier", FailClassifier)])

In [11]:
pipeline.fit()

TypeError: fit() missing 1 required positional argument: 'X'

In [ ]:
pipeline.predict(testgrid_data, grid, selected_dashboard, selected_job)

# Save Pipeline

In [ ]:
joblib.dump(pipeline, "model.joblib")

In [ ]:
# Sanity check to see if the saved model works locally
pipeline_loaded = joblib.load("model.joblib")
pipeline_loaded
pipeline_loaded.predict(build_logs[50:75])

In [ ]:
# Set credentials for your s3 storage
s3_endpoint_url = os.getenv("S3_ENDPOINT")
aws_access_key_id = os.getenv("S3_ACCESS_KEY")
aws_secret_access_key = os.getenv("S3_SECRET_KEY")
s3_bucket = os.getenv("S3_BUCKET")

In [ ]:
s3_resource = boto3.resource(
    "s3",
    endpoint_url=s3_endpoint_url,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)
bucket = s3_resource.Bucket(name=s3_bucket)

In [ ]:
# Upload your model
bucket.upload_file("model.joblib", "failure-classifier/classifier/model.joblib")

# Check if your model exists on s3
objects = [
    obj.key for obj in bucket.objects.filter(Prefix="") if "model.joblib" in obj.key
]
objects

# Test seldon deployment service 
We use the deployment [config](seldon_deployment_config.yaml) to deploy a seldon service.

In [ ]:
# Service url
# base_url = "http://failure-classifier-opf-seldon.apps.zero.massopen.cloud/predict"

In [ ]:
# Test set (same as locally checked model) testgrid_data, grid, selected_dashboard, selected_job
# test_list = 

In [ ]:
# convert the dataframe into a numpy array and then to a list (required by seldon)
data = {"data": {"ndarray": test_list}}

# create the query payload
json_data = json.dumps(data)
headers = {"content-Type": "application/json"}

# query our inference service
response = requests.post(base_url, data=json_data, headers=headers)
response

In [ ]:
response.json()

# Conclusion
In this notebook, we saw how to create and save a method for classifying flakey tests. We successfully deployed and tested the method utilizing S3 storage and a Seldon deployment on Openshift. 